In [2]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
import json
import math

C:\Users\juan-\AppData\Local\Temp\ipykernel_22892\2784272662.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df = pd.read_csv("Dati_sensori_meteo_20240209.csv")
df

,IdSensore,Data,Valore,idOperatore,Stato
0,14784,11/18/2023 06:50:00 PM,327.0,1,VA
1,14784,11/18/2023 07:00:00 PM,328.0,1,VA
2,14784,11/18/2023 07:10:00 PM,330.0,1,VA
3,14784,11/18/2023 07:20:00 PM,330.0,1,VA
4,14784,11/18/2023 07:30:00 PM,329.0,1,VA
...,...,...,...,...,...
37780091,5861,02/02/2024 11:00:00 PM,0.0,4,VA
37780092,5861,02/02/2024 11:10:00 PM,0.0,4,VA
37780093,5861,02/02/2024 11:20:00 PM,0.0,4,VA
37780094,5861,02/02/2024 11:30:00 PM,0.0,4,VA


In [4]:
# select columns
df = df[['IdSensore', 'Data', 'Valore', "idOperatore", "Stato"]]

# fix datetime
df['Data'] = pd.to_datetime(df['Data'], format="%m/%d/%Y %I:%M:%S %p", errors="coerce") #01/01/2023 12:00:00 AM

# Nulling stations that do not exist
df['Stato'].replace(np.nan, 'NA')

# fix values that break everything...
df['Stato'] = df['Stato'].replace(np.nan, 'NA')
df['Valore'] = df['Valore'].replace(-999.0, -9999.0)
df['Valore'] = df['Valore'].replace(999.0, -9999.0)
df['Valore'] = df['Valore'].replace(-777.0, 777.0)


In [5]:
df
#df.loc[df["Data"].dt == date(7,11,2023)]
#07/11/2023
#641363 with invalid status

,IdSensore,Data,Valore,idOperatore,Stato
0,14784,2023-11-18 18:50:00,327.0,1,VA
1,14784,2023-11-18 19:00:00,328.0,1,VA
2,14784,2023-11-18 19:10:00,330.0,1,VA
3,14784,2023-11-18 19:20:00,330.0,1,VA
4,14784,2023-11-18 19:30:00,329.0,1,VA
...,...,...,...,...,...
37780091,5861,2024-02-02 23:00:00,0.0,4,VA
37780092,5861,2024-02-02 23:10:00,0.0,4,VA
37780093,5861,2024-02-02 23:20:00,0.0,4,VA
37780094,5861,2024-02-02 23:30:00,0.0,4,VA


In [18]:
stations_df

,IdSensore,Tipologia,Unità DiMisura,IdStazione,NomeStazione,Quota,Provincia,DataStart,DataStop,Storico,UTM_Nord,UTM_Est,lng,lat,location
0,22006,Umidità Relativa,%,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
1,22007,Direzione Vento,°,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
2,22003,Temperatura,°C,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
3,22009,Velocità Vento,m/s,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
4,17612,Altezza Neve,cm,1545,San Siro Alpe Rescascìa,1285,CO,07/11/2019,NaN,N,5102069,518430,9.238323,46.071947,"(46.071947417956004, 9.238323435918229)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,10787,Velocità Vento,m/s,1266,Bertonico v.Moro,65,LO,12/04/2022,NaN,N,5009105,552297,9.666250,45.233494,"(45.23349364130374, 9.666250375296032)"
1244,19024,Radiazione Globale,W/m²,1422,Bergamo v.Maffei,249,BG,06/11/2013,NaN,N,5060404,552526,9.674653,45.695197,"(45.6951969397896, 9.674652773656335)"
1245,7048,Velocità Vento,m/s,695,Ponti sul Mincio v.San Martino,113,MN,19/12/2006,03/03/2021,S,5030184,631718,10.683357,45.412772,"(45.41277169168634, 10.683357343786835)"
1246,6990,Radiazione Globale,W/m²,679,Valmadrera v.Pozzi,247,LC,02/02/2006,NaN,N,5076576,527307,9.351658,45.842207,"(45.842207261022544, 9.351658166127958)"


In [13]:
unique_in_data = list(df.IdSensore.unique())

stations_df = pd.read_csv("Stazioni_Meteorologiche.csv")
unique_in_stations = list(stations_df.IdSensore.unique())

diffs = []
for u_d in unique_in_data:
    if u_d not in unique_in_stations:
        diffs.append(u_d)
diffs

[5983,
 22572,
 22573,
 22576,
 22579,
 22580,
 2491,
 2503,
 2611,
 4014,
 4022,
 4031,
 4039,
 5979,
 6044,
 6046,
 5981,
 6008,
 5987,
 5994,
 6029,
 6038,
 6063,
 6048,
 6788,
 6815,
 7025,
 6797,
 6825,
 22571,
 9862,
 9077,
 9761,
 9076,
 9328,
 9331,
 9741,
 9756,
 9757,
 9760,
 9758,
 9759,
 9763,
 9766,
 9765,
 9940,
 9931,
 10378,
 10557,
 10553,
 11756,
 11131,
 11132,
 11737,
 14082,
 11168,
 11167,
 11217,
 11818,
 11218,
 11734,
 11735,
 11736,
 11760,
 11739,
 4055,
 11757,
 11740,
 4062,
 4071,
 11742,
 11741,
 11743,
 4087,
 5126,
 11746,
 11745,
 11748,
 11747,
 11754,
 11749,
 11750,
 11752,
 11759,
 11755,
 11812,
 11758,
 11814,
 11763,
 11761,
 11815,
 11762,
 11811,
 11813,
 14083,
 14007,
 11817,
 14006,
 14031,
 14049,
 14065,
 14066,
 14363,
 14189,
 14113,
 14099,
 14100,
 14190,
 14232,
 14299,
 14216,
 14217,
 14237,
 14272,
 14251,
 14278,
 14576,
 14327,
 14358,
 14458,
 14382,
 14377,
 14411,
 14416,
 14509,
 14577,
 14459,
 14471,
 14472,
 14508,
 14554

In [19]:
print(f'Sensors in list {len(unique_in_stations)}')
print(f'Unique in data {len(unique_in_data)}')
print(f'Difference {len(diffs)}')

Sensors in list 1248
Unique in data 1220
Difference 208


In [20]:
df.loc[df.IdSensore == 5983]

,IdSensore,Data,Valore,idOperatore,Stato
3476,5983,2024-02-02 18:00:00,71.0,3,VA
3477,5983,2024-02-02 18:10:00,49.0,3,VA
3478,5983,2024-02-02 18:20:00,75.0,3,VA
3479,5983,2024-02-02 18:30:00,53.0,3,VA
3480,5983,2024-02-02 18:40:00,66.0,3,VA
...,...,...,...,...,...
37732405,5983,2024-02-02 17:10:00,241.0,3,VA
37732406,5983,2024-02-02 17:20:00,252.0,3,VA
37732407,5983,2024-02-02 17:30:00,252.0,3,VA
37732408,5983,2024-02-02 17:40:00,109.0,3,VA


In [25]:
data_copy = df.copy()
for sensor in diffs:
    print(f'removing {sensor}')
    data_copy = data_copy.loc[data_copy['IdSensore'] != sensor]


removing 5983
removing 22572
removing 22573
removing 22576
removing 22579
removing 22580
removing 2491
removing 2503
removing 2611
removing 4014
removing 4022
removing 4031
removing 4039
removing 5979
removing 6044
removing 6046
removing 5981
removing 6008
removing 5987
removing 5994
removing 6029
removing 6038
removing 6063
removing 6048
removing 6788
removing 6815
removing 7025
removing 6797
removing 6825
removing 22571
removing 9862
removing 9077
removing 9761
removing 9076
removing 9328
removing 9331
removing 9741
removing 9756
removing 9757
removing 9760
removing 9758
removing 9759
removing 9763
removing 9766
removing 9765
removing 9940
removing 9931
removing 10378
removing 10557
removing 10553
removing 11756
removing 11131
removing 11132
removing 11737
removing 14082
removing 11168
removing 11167
removing 11217
removing 11818
removing 11218
removing 11734
removing 11735
removing 11736
removing 11760
removing 11739
removing 4055
removing 11757
removing 11740
removing 4062
removing

,IdSensore,Data,Valore,idOperatore,Stato
0,14784,2023-11-18 18:50:00,327.0,1,VA
1,14784,2023-11-18 19:00:00,328.0,1,VA
2,14784,2023-11-18 19:10:00,330.0,1,VA
3,14784,2023-11-18 19:20:00,330.0,1,VA
4,14784,2023-11-18 19:30:00,329.0,1,VA
...,...,...,...,...,...
37780091,5861,2024-02-02 23:00:00,0.0,4,VA
37780092,5861,2024-02-02 23:10:00,0.0,4,VA
37780093,5861,2024-02-02 23:20:00,0.0,4,VA
37780094,5861,2024-02-02 23:30:00,0.0,4,VA


In [26]:
# 35331595 vs 37780096 
print(35331595 - 37780096)

-2448501


In [27]:
data_copy

,IdSensore,Data,Valore,idOperatore,Stato
0,14784,2023-11-18 18:50:00,327.0,1,VA
1,14784,2023-11-18 19:00:00,328.0,1,VA
2,14784,2023-11-18 19:10:00,330.0,1,VA
3,14784,2023-11-18 19:20:00,330.0,1,VA
4,14784,2023-11-18 19:30:00,329.0,1,VA
...,...,...,...,...,...
37780091,5861,2024-02-02 23:00:00,0.0,4,VA
37780092,5861,2024-02-02 23:10:00,0.0,4,VA
37780093,5861,2024-02-02 23:20:00,0.0,4,VA
37780094,5861,2024-02-02 23:30:00,0.0,4,VA


In [28]:
def arrange_timestamp(date_time):
    return str(date_time)

In [37]:
base_url_measurements = "http://127.0.0.1:8000/api/collections/MeteoMeasurement/"

batch_size = 1000000
batches = math.ceil(len(data_copy) / batch_size)
print(f"Number of batches: {batches}")

for i in range(0, batches):
    body = {
        "bulk": True,
        "items": []
    }
    
    i_start = i * batch_size
    i_end = len(data_copy) if len(data_copy) <= (batch_size * (i+1)) else (batch_size * (i+1))
    
    print(f'from {i_start} to {i_end}')
    timeseries_subset = data_copy.iloc[i_start:i_end]
    timeseries_subset = timeseries_subset[['IdSensore','Data','Valore','idOperatore','Stato']]
    timeseries_subset['Data'] = timeseries_subset['Data'].apply(arrange_timestamp)
    timeseries_subset = timeseries_subset.replace({np.nan: None})
    
    for i, sensor in timeseries_subset.iterrows():
        item = { "sensor_id_id": sensor.IdSensore, "date": sensor.Data, "value": sensor.Valore , 'status': sensor.Stato, 'operative_id': sensor.idOperatore}
        body["items"].append(item)
        
    body_json = json.dumps(body)
    body_json
    req = requests.post(base_url_measurements, body_json)
    if req.status_code == 500:
        print(req)
    print(f'request {i} - {req.status_code}')

Number of batches: 36
from 0 to 1000000
request 1011456 - 200
from 1000000 to 2000000
request 2087470 - 200
from 2000000 to 3000000


KeyboardInterrupt: 